In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
import re 
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import pandas_datareader.data as web

In [2]:
datafile = 'Oct18-Oct19.csv'
data = pd.read_csv(datafile,encoding="ISO-8859-1")
data1 = data.copy()

data1["tweet_influence"] = data1['retweets']*0.6+data1['favorites']*0.3+data1['replies']*0.1


In [3]:
data1 = data1.sort_values(by=['tweet_influence'],ascending=False)


In [10]:
inf_30 = data1[:30]
inf_30

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,tweet_influence
4023258,2/24/19 17:30,Nike,NaN,5992,192317,448295,Show them what crazy dreams can do. #justdoitp...,NaN,NaN,#justdoitpic,1.100000e+18,https://twitter.com/Nike/status/10997231843691...,250477.9
1368623,7/7/19 16:56,Nike,NaN,7616,96619,278856,This team wins. Everyone wins. Victory is when...,NaN,@USWNTpic,#justdoit,1.150000e+18,https://twitter.com/Nike/status/11479122016524...,142389.8
1260767,7/4/19 1:24,BlairImani,NaN,2065,83220,245166,White American get more upset about Nike and m...,NaN,NaN,NaN,1.150000e+18,https://twitter.com/BlairImani/status/11465904...,123688.3
647858,6/1/19 13:59,Nike,NaN,5410,72668,185598,DonâÃÃ´t change your dream. Change the worl...,NaN,NaN,#justdoitpic,1.130000e+18,https://twitter.com/Nike/status/11348218579059...,99821.2
334053,5/2/19 6:21,JadenOrJ,NaN,384,67137,179462,"Nike did it again, I love this pic.twitter.com...",NaN,NaN,NaN,1.120000e+18,https://twitter.com/JadenOrJ/status/1123834983...,94159.2
2762222,10/10/18 0:18,hellahellish,NaN,1387,43483,208822,guys think itâÃÃ´s âÃÃºgayâÃÃ¹ to wea...,NaN,NaN,NaN,1.049820e+18,https://twitter.com/hellahellish/status/104981...,88875.1
2248894,10/22/19 20:12,TheJayBurns,NaN,520,42093,189602,We not 16 years old you not wearing my hoodies...,NaN,NaN,NaN,1.190000e+18,https://twitter.com/TheJayBurns/status/1186737...,82188.4
2768159,10/10/18 16:14,SInow,NaN,2533,60417,143709,"Justin Gallegos, a runner at Oregon with cereb...",NaN,@kabdullah360,NaN,1.050060e+18,https://twitter.com/SInow/status/1050057048686...,79616.2
735177,6/9/19 16:14,Nike,NaN,879,45042,114434,Crazy dreams take crazy effort. 18 career majo...,NaN,@RafaelNadal,#justdoitpic,1.140000e+18,https://twitter.com/Nike/status/11377548729025...,61443.3
916313,6/27/19 18:37,Eugene3005,NaN,517,35386,118341,12 year old me creating $300 NikeID soccer boo...,NaN,NaN,NaN,1.140000e+18,https://twitter.com/Eugene3005/status/11443139...,56785.6


In [13]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            self.data_model = inf_30
            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            self.data = inf_30
            #self.data = self.data[['date','text']]
        else:
            self.data = inf_30
            not_null_text = 1 ^ pd.isnull(self.data["text"])
            not_null_date = 1 ^ pd.isnull(self.data["date"])
            self.data = self.data.loc[not_null_date & not_null_text, :]

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None

In [14]:
data = TwitterData_Initialize()
data.initialize()
data.processed_data.head(5)

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,tweet_influence
4023258,2/24/19 17:30,Nike,NaN,5992,192317,448295,Show them what crazy dreams can do. #justdoitp...,NaN,NaN,#justdoitpic,1.100000e+18,https://twitter.com/Nike/status/10997231843691...,250477.9
1368623,7/7/19 16:56,Nike,NaN,7616,96619,278856,This team wins. Everyone wins. Victory is when...,NaN,@USWNTpic,#justdoit,1.150000e+18,https://twitter.com/Nike/status/11479122016524...,142389.8
1260767,7/4/19 1:24,BlairImani,NaN,2065,83220,245166,White American get more upset about Nike and m...,NaN,NaN,NaN,1.150000e+18,https://twitter.com/BlairImani/status/11465904...,123688.3
647858,6/1/19 13:59,Nike,NaN,5410,72668,185598,DonâÃÃ´t change your dream. Change the worl...,NaN,NaN,#justdoitpic,1.130000e+18,https://twitter.com/Nike/status/11348218579059...,99821.2
334053,5/2/19 6:21,JadenOrJ,NaN,384,67137,179462,"Nike did it again, I love this pic.twitter.com...",NaN,NaN,NaN,1.120000e+18,https://twitter.com/JadenOrJ/status/1123834983...,94159.2


In [15]:
class TwitterCleanuper:
    def iterate(self):
        for cleanup_method in [self.remove_urls,
                               self.remove_usernames,
                               self.remove_na,
                               self.remove_special_chars,
                               self.remove_numbers]:
            yield cleanup_method

    @staticmethod
    def remove_by_regex(tweets, regexp):
        tweets.loc[:, "text"].replace(regexp, "", inplace=True)
        return tweets

    def remove_urls(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"http.?://[^\s]+[\s]?"))

    def remove_na(self, tweets):
        return tweets[tweets["text"] != "Not Available"]

    def remove_special_chars(self, tweets):  # it unrolls the hashtags to normal words
        for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
            tweets.loc[:, "text"].replace(remove, "", inplace=True)
        return tweets

    def remove_usernames(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"@[^\s]+[\s]?"))

    def remove_numbers(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"\s?[0-9]+\.?[0-9]*"))

In [16]:
class TwitterData_Cleansing(TwitterData_Initialize):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    def cleanup(self, cleanuper):
        t = self.processed_data
        for cleanup_method in cleanuper.iterate():
            if not self.is_testing:
                t = cleanup_method(t)
            else:
                if cleanup_method.__name__ != "remove_na":
                    t = cleanup_method(t)

        self.processed_data = t

In [23]:
data = TwitterData_Cleansing(data)
data.cleanup(TwitterCleanuper())
# data.processed_data.head(10)

NameError: name 'regex' is not defined

In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
word_tokens = []
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siyanchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/siyanchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
for index in data.processed_data.iloc[:,6]:
    word_tokens.append(word_tokenize(index))

print('done')



done


In [20]:
filtered_sentence = [] 


for word in word_tokens:
    this_filtered_sentence = [w.lower() for w in word if not w.lower() in stop_words] 
    filtered_sentence.append(this_filtered_sentence)

In [21]:

data.processed_data['filtered']=filtered_sentence

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
data.processed_data

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,tweet_influence,filtered
4023258,2/24/19 17:30,Nike,NaN,5992,192317,448295,Show them what crazy dreams can do. #justdoitp...,NaN,NaN,#justdoitpic,1.100000e+18,https://twitter.com/Nike/status/10997231843691...,250477.9,"[show, crazy, dreams, ., #, justdoitpic.twitte..."
1368623,7/7/19 16:56,Nike,NaN,7616,96619,278856,This team wins. Everyone wins. Victory is when...,NaN,@USWNTpic,#justdoit,1.150000e+18,https://twitter.com/Nike/status/11479122016524...,142389.8,"[team, wins, ., everyone, wins, ., victory, wi..."
1260767,7/4/19 1:24,BlairImani,NaN,2065,83220,245166,White American get more upset about Nike and m...,NaN,NaN,NaN,1.150000e+18,https://twitter.com/BlairImani/status/11465904...,123688.3,"[white, american, get, upset, nike, mermaids, ..."
647858,6/1/19 13:59,Nike,NaN,5410,72668,185598,DonâÃÃ´t change your dream. Change the worl...,NaN,NaN,#justdoitpic,1.130000e+18,https://twitter.com/Nike/status/11348218579059...,99821.2,"[donâãã´t, change, dream, ., change, world,..."
334053,5/2/19 6:21,JadenOrJ,NaN,384,67137,179462,"Nike did it again, I love this pic.twitter.com...",NaN,NaN,NaN,1.120000e+18,https://twitter.com/JadenOrJ/status/1123834983...,94159.2,"[nike, ,, love, pic.twitter.com/u7u0awwosr]"
2762222,10/10/18 0:18,hellahellish,NaN,1387,43483,208822,guys think itâÃÃ´s âÃÃºgayâÃÃ¹ to wea...,NaN,NaN,NaN,1.049820e+18,https://twitter.com/hellahellish/status/104981...,88875.1,"[guys, think, itâãã´s, âããºgayâãã¹, w..."
2248894,10/22/19 20:12,TheJayBurns,NaN,520,42093,189602,We not 16 years old you not wearing my hoodies...,NaN,NaN,NaN,1.190000e+18,https://twitter.com/TheJayBurns/status/1186737...,82188.4,"[16, years, old, wearing, hoodies, keeping, em..."
2768159,10/10/18 16:14,SInow,NaN,2533,60417,143709,"Justin Gallegos, a runner at Oregon with cereb...",NaN,@kabdullah360,NaN,1.050060e+18,https://twitter.com/SInow/status/1050057048686...,79616.2,"[justin, gallegos, ,, runner, oregon, cerebral..."
735177,6/9/19 16:14,Nike,NaN,879,45042,114434,Crazy dreams take crazy effort. 18 career majo...,NaN,@RafaelNadal,#justdoitpic,1.140000e+18,https://twitter.com/Nike/status/11377548729025...,61443.3,"[crazy, dreams, take, crazy, effort, ., 18, ca..."
916313,6/27/19 18:37,Eugene3005,NaN,517,35386,118341,12 year old me creating $300 NikeID soccer boo...,NaN,NaN,NaN,1.140000e+18,https://twitter.com/Eugene3005/status/11443139...,56785.6,"[12, year, old, creating, $, 300, nikeid, socc..."


In [24]:
sentiment= SentimentIntensityAnalyzer()

data.processed_data['polarity_value']="Default"
data.processed_data['neg']=0.0
data.processed_data['pos']=0.0
data.processed_data['neu']=0.0
data.processed_data['compound']=0.0
for index,row in data.processed_data.iterrows():
    senti = sentiment.polarity_scores(row['text'])
    data.processed_data.set_value(index,'polarity_value',senti)
    data.processed_data.set_value(index,'neg',senti['neg'])
    data.processed_data.set_value(index,'pos',senti['pos'])
    data.processed_data.set_value(index,'neu',senti['neu'])
    data.processed_data.set_value(index,'compound',senti['compound'])
data.processed_data[:10]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,tweet_influence,filtered,polarity_value,neg,pos,neu,compound
4023258,2/24/19 17:30,Nike,NaN,5992,192317,448295,Show them what crazy dreams can do. #justdoitp...,NaN,NaN,#justdoitpic,1.100000e+18,https://twitter.com/Nike/status/10997231843691...,250477.9,"[show, crazy, dreams, ., #, justdoitpic.twitte...","{'neg': 0.216, 'neu': 0.541, 'pos': 0.243, 'co...",0.216,0.243,0.541,0.0772
1368623,7/7/19 16:56,Nike,NaN,7616,96619,278856,This team wins. Everyone wins. Victory is when...,NaN,@USWNTpic,#justdoit,1.150000e+18,https://twitter.com/Nike/status/11479122016524...,142389.8,"[team, wins, ., everyone, wins, ., victory, wi...","{'neg': 0.081, 'neu': 0.541, 'pos': 0.378, 'co...",0.081,0.378,0.541,0.8689
1260767,7/4/19 1:24,BlairImani,NaN,2065,83220,245166,White American get more upset about Nike and m...,NaN,NaN,NaN,1.150000e+18,https://twitter.com/BlairImani/status/11465904...,123688.3,"[white, american, get, upset, nike, mermaids, ...","{'neg': 0.171, 'neu': 0.829, 'pos': 0.0, 'comp...",0.171,0.000,0.829,-0.4391
647858,6/1/19 13:59,Nike,NaN,5410,72668,185598,DonâÃÃ´t change your dream. Change the worl...,NaN,NaN,#justdoitpic,1.130000e+18,https://twitter.com/Nike/status/11348218579059...,99821.2,"[donâãã´t, change, dream, ., change, world,...","{'neg': 0.0, 'neu': 0.778, 'pos': 0.222, 'comp...",0.000,0.222,0.778,0.2500
334053,5/2/19 6:21,JadenOrJ,NaN,384,67137,179462,"Nike did it again, I love this pic.twitter.com...",NaN,NaN,NaN,1.120000e+18,https://twitter.com/JadenOrJ/status/1123834983...,94159.2,"[nike, ,, love, pic.twitter.com/u7u0awwosr]","{'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'comp...",0.000,0.412,0.588,0.6369
2762222,10/10/18 0:18,hellahellish,NaN,1387,43483,208822,guys think itâÃÃ´s âÃÃºgayâÃÃ¹ to wea...,NaN,NaN,NaN,1.049820e+18,https://twitter.com/hellahellish/status/104981...,88875.1,"[guys, think, itâãã´s, âããºgayâãã¹, w...","{'neg': 0.268, 'neu': 0.685, 'pos': 0.048, 'co...",0.268,0.048,0.685,-0.8912
2248894,10/22/19 20:12,TheJayBurns,NaN,520,42093,189602,We not 16 years old you not wearing my hoodies...,NaN,NaN,NaN,1.190000e+18,https://twitter.com/TheJayBurns/status/1186737...,82188.4,"[16, years, old, wearing, hoodies, keeping, em...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.000,1.000,0.0000
2768159,10/10/18 16:14,SInow,NaN,2533,60417,143709,"Justin Gallegos, a runner at Oregon with cereb...",NaN,@kabdullah360,NaN,1.050060e+18,https://twitter.com/SInow/status/1050057048686...,79616.2,"[justin, gallegos, ,, runner, oregon, cerebral...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.000,1.000,0.0000
735177,6/9/19 16:14,Nike,NaN,879,45042,114434,Crazy dreams take crazy effort. 18 career majo...,NaN,@RafaelNadal,#justdoitpic,1.140000e+18,https://twitter.com/Nike/status/11377548729025...,61443.3,"[crazy, dreams, take, crazy, effort, ., 18, ca...","{'neg': 0.224, 'neu': 0.467, 'pos': 0.308, 'co...",0.224,0.308,0.467,0.4215
916313,6/27/19 18:37,Eugene3005,NaN,517,35386,118341,12 year old me creating $300 NikeID soccer boo...,NaN,NaN,NaN,1.140000e+18,https://twitter.com/Eugene3005/status/11443139...,56785.6,"[12, year, old, creating, $, 300, nikeid, socc...","{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'comp...",0.000,0.115,0.885,0.2960


In [25]:
data1['text'][200692]
data1['text'][90715]

'Check out what I just added to my closet on Poshmark: New! Nike Magenta Free 7 Trainers Sneakers | 7.5. https://poshmark.com/listing/5cac691741950185eb1ebe2b?utm_campaign=referral_code%3DSPARKDAYS&utm_content=feature%3Dsh_li_cl_web%26rfuid%3D59a118521345e03aae0097be%26ext_trk%3Dbranch&utm_source=tw_sh â\x80\x9aÃ\x84Â¶ via @poshmarkapp #shopmycloset'

# negative comment from influential twitter users

In [26]:
data.processed_data[data.processed_data["compound"]<0]['text']

1260767    White American get more upset about Nike and m...
2762222    guys think itâÃÃ´s âÃÃºgayâÃÃ¹ to wea...
3083406    Socks Fresh pack of white tees Nike Tech Doubl...
1641847    WAIT LMAODKFJSLS sheâÃÃ´s wearing nikes to ...
1076253    Race Baiting Colin Kaepernick said the America...
2438296    So let me get this straight: many Trump suppor...
675536     If these homophobic ass bitches think they can...
1232575    ItâÃÃ´ll be tough but if Nike wants to get ...
Name: text, dtype: object

In [27]:
print ("List index-value are : ") 
for index, value in zip(range(len(data.processed_data)), data.processed_data): 
    print (index, value) 

List index-value are : 
0 date
1 username
2 to
3 replies
4 retweets
5 favorites
6 text
7 geo
8 mentions
9 hashtags
10 id
11 permalink
12 tweet_influence
13 filtered
14 polarity_value
15 neg
16 pos
17 neu
18 compound


# hashtag frequency 


In [28]:
###### hashtag

#data1[data1["username"].str.match('com')] 
Nike_account_data = data1[(data1["username"]=='Nike' )| (data1["username"] == 'NikeNYC' )| (data1["username"] =='NikeLA')]
official_hashtag = Nike_account_data[pd.notnull(Nike_account_data['hashtags'])]

#official_hashtag = official_hashtag['hashtags'].str.split(" ", n=1, expand=True)

#official_hashtag[0].append(official_hashtag[1])
#Nike_account_data[pd.notnull(Nike_account_data['hashtags'])]

official_hashtag.head(10)



,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,tweet_influence
4023258,2/24/19 17:30,Nike,NaN,5992,192317,448295,Show them what crazy dreams can do. #justdoitp...,NaN,NaN,#justdoitpic,1.100000e+18,https://twitter.com/Nike/status/10997231843691...,250477.9
1368623,7/7/19 16:56,Nike,NaN,7616,96619,278856,This team wins. Everyone wins. Victory is when...,NaN,@USWNTpic,#justdoit,1.150000e+18,https://twitter.com/Nike/status/11479122016524...,142389.8
647858,6/1/19 13:59,Nike,NaN,5410,72668,185598,DonâÃÃ´t change your dream. Change the worl...,NaN,NaN,#justdoitpic,1.130000e+18,https://twitter.com/Nike/status/11348218579059...,99821.2
735177,6/9/19 16:14,Nike,NaN,879,45042,114434,Crazy dreams take crazy effort. 18 career majo...,NaN,@RafaelNadal,#justdoitpic,1.140000e+18,https://twitter.com/Nike/status/11377548729025...,61443.3
3154612,11/18/18 18:07,Nike,NaN,355,17520,53284,"ItâÃÃ´s only crazy until you do it, and @ob...",NaN,@obj,#justdoitpic,1.064220e+18,https://twitter.com/Nike/status/10642185970425...,26532.7
436182,5/12/19 14:01,Nike,NaN,945,13983,45546,Having a dream wonâÃÃ´t change the world. S...,NaN,NaN,#justdoitpic,1.130000e+18,https://twitter.com/Nike/status/11275744554949...,22148.1
2868749,10/18/18 16:00,Nike,NaN,277,11069,47741,Kids from Akron donâÃÃ´t just dream it. The...,NaN,NaN,#justdoitpic,1.052950e+18,https://twitter.com/Nike/status/10529525460235...,20991.4
1430299,7/10/19 5:00,Nike,NaN,6560,5439,37092,Sport has the power to change your life. Sport...,NaN,NaN,#SportChangesEverythingpic,1.150000e+18,https://twitter.com/Nike/status/11488191401964...,15047.0
3191160,11/22/18 15:19,Nike,NaN,253,8062,27265,"âÃÃºMy dad sacrificed his dream, so I could...",NaN,@KyrieIrving,#justdoitpic,1.065630e+18,https://twitter.com/Nike/status/10656257255442...,13042.0
352019,5/3/19 21:47,Nike,NaN,219,7077,19872,"Never slow down for the world, one day it will...",NaN,NaN,#justdoitpic,1.120000e+18,https://twitter.com/Nike/status/11244303028461...,10229.7


In [29]:
official_hashtag = official_hashtag['hashtags'].str.split(" ", n=1, expand=True)


In [30]:
x = pd.DataFrame(official_hashtag[0].append(official_hashtag[1]).dropna())
x.columns = (['hashtag'])
x

,hashtag
4023258,#justdoitpic
1368623,#justdoit
647858,#justdoitpic
735177,#justdoitpic
3154612,#justdoitpic
...,...
264858,#NikeReact
246492,#NikeReact
4124131,#NikeReact
314218,#NikeReact


In [31]:
y = x['hashtag'].value_counts()
y

#justdoit                     25
#justdoitpic                  22
#GoMoreGetMore                 6
#NikeReact                     5
#af1                           4
#allfor1                       4
#af1pic                        4
#TheGrailpic                   3
#AirMaxDay                     3
#thereplay                     3
#madetoplay                    2
#NikeOnAir                     2
#chicagomarathonpic            2
#LaMezcla                      2
#nikeadaptpic                  2
#madetoplaypic                 2
#chicagomarathon               2
#nikeadapt                     1
#nordstromxnikepic             1
#OpeningDayLA                  1
#carharttwippic                1
#nikebymelrosepic              1
#BeTrue                        1
#enjoytherun                   1
#gomoregetmore                 1
#UntilWeAllWin                 1
#nikereactpic                  1
#SportChangesEverythingpic     1
#NikeOnAirpic                  1
#contact                       1
#sportchan

In [32]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
hashcount = pd.DataFrame(y)
hashcount[hashcount['hashtag']>1]
ax = hashcount[hashcount['hashtag']>1].plot.barh()

#### KNN cluster    (not work)

In [34]:
# Importing libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import numpy as np
from collections import Counter

In [35]:
train_data=data1[1:50]

In [36]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
 
# Cleaning the text sentences so that punctuation marks, stop words &amp; digits are removed
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","",normalized)
    y = processed.split()
    return y

In [37]:
train_clean_sentences = []
for line in train_data['text']:
    line = line.strip()
    cleaned = clean(line)
    cleaned = ' '.join(cleaned)
    train_clean_sentences.append(cleaned)
 
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(train_clean_sentences)

In [38]:
# Creating true labels for 30 training sentences
y_train = np.zeros(49)
y_train[10:20] = 1
y_train[20:50] = 2

In [40]:
# Clustering the document with KNN classifier
modelknn = KNeighborsClassifier(n_neighbors=5)
modelknn.fit(X,y_train)
 
# Clustering the training 30 sentences with K-means technique
modelkmeans = KMeans(n_clusters=3, init='k-means++', max_iter=200, n_init=100)
modelkmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=200,
    n_clusters=3, n_init=100, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [41]:
test_sentences = y
test_clean_sentence = []
for test in test_sentences:
    cleaned_test = clean(test)
    cleaned = ' '.join(cleaned_test)
    cleaned = re.sub(r"\d+","",cleaned)
    test_clean_sentence.append(cleaned)
 
Test = vectorizer.transform(test_clean_sentence)
 
true_test_labels = ['ship','sad','angry']
predicted_labels_knn = modelknn.predict(Test)
predicted_labels_kmeans = modelkmeans.predict(Test)
 
print ("\nBelow 3 sentences will be predicted against the learned nieghbourhood and learned clusters:\n1. ",\
test_sentences[0],"\n2. ",test_sentences[1],"\n3. ",test_sentences[2])
print ("\n-------------------------------PREDICTIONS BY KNN------------------------------------------")
print ("\n",test_sentences[0],":",true_test_labels[np.int(predicted_labels_knn[0])],\
"\n",test_sentences[1],":",true_test_labels[np.int(predicted_labels_knn[1])],\
"\n",test_sentences[2],":",true_test_labels[np.int(predicted_labels_knn[2])])
 
print ("\n-------------------------------PREDICTIONS BY K-Means--------------------------------------")
print ("\nIndex of happy cluster : ",Counter(modelkmeans.labels_[0:10]).most_common(1)[0][0])
print ("Index of fear cluster : ",Counter(modelkmeans.labels_[10:20]).most_common(1)[0][0])
print ("Index of angry cluster : ",Counter(modelkmeans.labels_[20:30]).most_common(1)[0][0])
print ("\n",test_sentences[0],":",predicted_labels_kmeans[0],\
"\n",test_sentences[1],":",predicted_labels_kmeans[1],\
"\n",test_sentences[2],":",predicted_labels_kmeans[2])

AttributeError: 'int' object has no attribute 'lower'

In [42]:
x = pd.DataFrame([data1[60:70]['text']])
x

,593468,1513582,3861812,2002462,1987583,156875,4035534,885050,4066222,583038
text,Breaking: Ja Morant @igotgame_12 announced heâ...,Jorja Smith for Nike Women pic.twitter.com/Qgj...,Thanks @Nike pic.twitter.com/9Rqm4M4Cyr,not believing this until i see them in matchin...,Nike Air Force 1 Reflective pic.twitter.com/PS...,"FIRST LOOK: @TigerWoods wins Masters. Nike, on...",Nike and Serena Williams sent a powerful messa...,Nike Air Force 1 x Tupac Shakur pic.twitter.co...,"13-year-old Olivia Moultrie turns pro, will si...",Let the Roland Garros begin. Here is my French...


In [44]:
y = x.values.tolist()[0]
y

['Breaking: Ja Morant @igotgame_12 announced heâ\x80\x9aÃ\x84Ã´s signing with Nike. Itâ\x80\x9aÃ\x84Ã´s a multi-year deal. pic.twitter.com/ssJ6sT3rB6',
 'Jorja Smith for Nike Women pic.twitter.com/Qgj0ZkqCfF',
 'Thanks @Nike pic.twitter.com/9Rqm4M4Cyr',
 'not believing this until i see them in matching nike tech suits https://twitter.com/MirrorCeleb/status/1178793141173927937 â\x80\x9aÃ\x84Â¶',
 'Nike Air Force 1 Reflective pic.twitter.com/PSobWaoA1r',
 'FIRST LOOK: @TigerWoods wins Masters. Nike, one of two companies to stand by him, releases "Tiger Woods: Same Dream" spot. pic.twitter.com/tojz8cdaLk',
 "Nike and Serena Williams sent a powerful message about 'crazy' women during the 2019 #Oscars â\x80\x9aÃ\x84Ã® Here's the full commercialpic.twitter.com/uRn6zCfKjp",
 'Nike Air Force 1 x Tupac Shakur pic.twitter.com/RzxjdpludW',
 '13-year-old Olivia Moultrie turns pro, will sign a contract with Nike (via @COPA90US)pic.twitter.com/zvlmoQ9cF8',
 'Let the Roland Garros begin. Here is my F

In [45]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/siyanchen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [46]:
data.processed_data
train_clean_sentences = []
for line in data.processed_data['filtered']:
    for x in line:
        train_clean_sentences.append(x)
train_clean_sentences 

['show',
 'crazy',
 'dreams',
 '.',
 '#',
 'justdoitpic.twitter.com/3fo2xmvkbt',
 'team',
 'wins',
 '.',
 'everyone',
 'wins',
 '.',
 'victory',
 'win',
 '.',
 "'s",
 'crazy',
 '.',
 '#',
 'justdoit',
 '@',
 'uswntpic.twitter.com/pbu7ue2ies',
 'white',
 'american',
 'get',
 'upset',
 'nike',
 'mermaids',
 'concentration',
 'camps',
 '.',
 'donâ\x80\x9aã\x84ã´t',
 'change',
 'dream',
 '.',
 'change',
 'world',
 '.',
 '#',
 'justdoitpic.twitter.com/0cj1ztpyvn',
 'nike',
 ',',
 'love',
 'pic.twitter.com/u7u0awwosr',
 'guys',
 'think',
 'itâ\x80\x9aã\x84ã´s',
 'â\x80\x9aã\x84ãºgayâ\x80\x9aã\x84ã¹',
 'wear',
 'black',
 'nail',
 'polish',
 ',',
 'eyeshadow',
 ',',
 'rings',
 ',',
 'etc',
 '...',
 '..',
 '...',
 '..',
 'yall',
 'know',
 'much',
 'pussy',
 'yall',
 'could',
 'get',
 'yall',
 'shit',
 '?',
 'promise',
 'nothing',
 'sexy',
 'boring',
 'ass',
 'nike',
 'outfit',
 '16',
 'years',
 'old',
 'wearing',
 'hoodies',
 'keeping',
 'em',
 '.',
 'nike',
 'hoodie',
 '$',
 '100',
 'foh',
 'i

In [47]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(train_clean_sentences)

In [48]:
X

<568x332 sparse matrix of type '<class 'numpy.float64'>'
	with 491 stored elements in Compressed Sparse Row format>

In [49]:
y_train = np.zeros(424)
y_train[10:20] = 1
y_train[20:424] = 2


In [50]:
#modelknn = KNeighborsClassifier(n_neighbors=5)
#modelknn.fit(X,y_train)

ValueError: Found input variables with inconsistent numbers of samples: [568, 424]

In [51]:
#modelkmeans = KMeans(n_clusters=3, init='k-means++', max_iter=200, n_init=100)
#modelkmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=200,
    n_clusters=3, n_init=100, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

# data preprocessing for pyspark

In [54]:
data_mention=data1[:3000][['username','mentions']].dropna()

In [55]:
data_mention['mentions'] = data_mention['mentions'].apply(lambda x: x.split(' '))
data_mention = data_mention.explode('mentions')

In [56]:
data_mention

,username,mentions
5425,brkicks,@J23apppic
6241,ComplexSports,@obj
6241,ComplexSports,@God_Son80
6241,ComplexSports,@Nike
6253,darrenrovell,@obj
...,...,...
4412073,Cristiano,@KMbappepic
4427858,brkicks,@KyrieIrving
4427858,brkicks,@trvisXX
4427858,brkicks,@Dloading


In [57]:
#cdata_mention.to_csv(r'mention2.csv')

In [3]:
dfp = pd.read_csv("mention2.csv")

In [4]:
dfp['mentions']=dfp['mentions'].str.replace(r'@','')

In [7]:
dfp.to_csv(r'metions4.csv')

,Unnamed: 0,username,mentions
0,5425,brkicks,J23apppic
1,6241,ComplexSports,obj
2,6241,ComplexSports,God_Son80
3,6241,ComplexSports,Nike
4,6253,darrenrovell,obj
...,...,...,...
1396,4412073,Cristiano,KMbappepic
1397,4427858,brkicks,KyrieIrving
1398,4427858,brkicks,trvisXX
1399,4427858,brkicks,Dloading


In [ ]:
df_jordan_positive.to_csv(r'df_jordan_pos.csv')

In [ ]:
stop = stopwords.words('english')